In [1]:
import girderformindlogger
import girder_client as gc
import pandas as pd
from bson.objectid import ObjectId
from datetime import datetime 
from girderformindlogger.constants import AccessType
from girderformindlogger.utility.response import isodatetime, delocalize, aggregateAndSave, _oneResponsePerDate, last7Days
from girderformindlogger.models.response_folder import ResponseItem
from pymongo import DESCENDING

In [2]:
girder = gc.GirderClient(host="dev.mindlogger.org")
girder.authenticate(interactive=True)

In [7]:
resp = girder.get('response')

In [60]:
girder.get('applet/{}'.format('5d6ee74a8f66701eb8166011'))

{'activities': {'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/MindLoggerDemo/MindLoggerDemo_schema': {'@context': ['https://raw.githubusercontent.com/ReproNim/schema-standardization/master/contexts/generic',
    'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/MindLoggerDemo/MindLoggerDemo_context'],
   '@id': '/MindLoggerDemo_schema',
   '@type': ['https://raw.githubusercontent.com/ReproNim/schema-standardization/master/schemas/Activity'],
   '_id': 'activity/5d6ee74f8f66701eb8166017',
   'http://schema.org/description': [{'@language': 'en',
     '@value': 'This activity contains one of each type of widget supported by the MindLogger mobile app.'}],
   'http://schema.org/schemaVersion': [{'@language': 'en', '@value': '0.0.1'}],
   'http://schema.org/version': [{'@language': 'en', '@value': '0.0.1'}],
   'http://schema.repronim.org/preamble': [{'@language': 'en',
     '@value': 'Please follow the instructions fo

In [ ]:
[response.get('meta', {}).get('responses', {}) for response in resp]

In [ ]:
def formatResponse(response):
    metadata = response.get('meta', response)
    thisResponse = {
        "thisResponse": {
            "schema:startDate": isodatetime(
                metadata.get('responseStarted')
            ),
            "schema:endDate": isodatetime(
                metadata.get('responseCompleted')
            ),
            "responses": {itemURI: metadata['responses'][itemURI] for itemURI in metadata.get('responses', {})}
        }#,
#           "allToDate": {
#             "schema:startDate": schema:DateTime,
#             "schema:endDate": schema:DateTime,
#             "schema:duration": schema:Duration,
#             itemURI*: [
#               {
#                 "value": any, //can be however its represented in schema, but usually numeric
#                 "count": numeric,
#               },
#             ],
#           },
#           "last7Days": { // from response time
#             "schema:startDate": schema:DateTime,
#             "schema:endDate": schema:DateTime,
#             "schema:duration": schema:Duration,
#             itemURI*: [
#               {
#                 "value": any, //can be however its represented in schema, but usually numeric
#                 "date": isostring,
#               },
#             ],
#           }
    }
    return(thisResponse)

In [ ]:
singleResponse = formatResponse(resp[0])
singleResponse

In [53]:
def last7Days(appletId, informant, reviewer, subject=None, referenceDate=None):
    referenceDate = delocalize(
        datetime.now() if referenceDate is None else referenceDate # TODO allow timeless dates
    )
#     print(applet)
#     print(referenceDate)
#     print(type(referenceDate))
    print({
            "baseParentType": 'user',
            "baseParentId": {
                "$in": [
                    reviewer.get('_id'),
                    ObjectId(reviewer.get('_id'))
                ]
            },
            "created": {
                "$lte": "ISODate({})".format(referenceDate.isoformat())
            },
            "meta.applet.@id": {
                "$in": [
                    appletId,
                    ObjectId(appletId)
                ]
            }
        })
    latestResponse = list(ResponseItem().find(
        query={
            "baseParentType": 'user',
            "baseParentId": reviewer.get('_id'),
            "created": {
                "$lte": referenceDate
            },
            "meta.applet.@id": appletId
        },
        sort=[("created", DESCENDING)],
        user=reviewer,
        level=AccessType.READ
    ))
    print(latestResponse) 
    latestResponse = list(ResponseItem().find(
        query={
            "baseParentType": 'user',
            "baseParentId": ObjectId(informant),
#             "created": {
#                 "$lte": referenceDate
#             },
#             "meta.applet.@id": {
#                 "$in": [
#                     appletId,
#                     ObjectId(appletId)
#                 ]
#             }
        },
        sort=[("created", DESCENDING)],
        force=True
    ))
    print(latestResponse)
    latestResponse = latestResponse[0] if len(latestResponse) else {}
    metadata = latestResponse.get('meta', {})
    if "last7Days" not in metadata or "allTime" not in metadata:
        latestResponse = aggregateAndSave(latestResponse, reviewer)
    l7d = latestResponse.get('meta', {}).get('last7Days', {})
    l7d["responses"] = _oneResponsePerDate(l7d.get("responses", {})) if l7d is not None else {}
    return(l7d)

In [61]:
girder.get('context')

{'@context': {'@base': 'https://mindlogger-dev.vasegurt.com/api/v1',
  '@language': 'en-US',
  'prov': 'http://www.w3.org/ns/prov#',
  'schema': 'http://schema.org/'}}

In [55]:
last7Days('5d6ee74a8f66701eb8166011', '5b59f003ad97094c99a10c4a', girder.get('user/me'))

{'baseParentType': 'user', 'baseParentId': {'$in': ['5c3cc859dedfea0fb67ee872', ObjectId('5c3cc859dedfea0fb67ee872')]}, 'created': {'$lte': 'ISODate(2019-09-18T15:44:32.444361)'}, 'meta.applet.@id': {'$in': ['5d6ee74a8f66701eb8166011', ObjectId('5d6ee74a8f66701eb8166011')]}}
[]
[{'_id': ObjectId('5b59f02dad97094c99a11006'), 'baseParentType': 'user', 'creatorId': ObjectId('5b1830ddad97092bdb8e99d1'), 'meta': {'devices:os': 'devices:None', 'responses': [{'schema:name': {'@value': 'IAT_01', '@language': 'en-US'}, 'choice': [{'option_text': {'@value': 'Occasionally', '@language': 'en-US'}, 'value': '2'}], 'prompt': {'@value': 'Based upon the following five-point likert scale, select the response that best represents the frequency of the behavior described in the following 20-item questionnaire: How often do you find that you stay on-line longer than you intended?', '@language': 'en-US'}, 'time': 1519323810800}, {'schema:name': {'@value': 'IAT_02', '@language': 'en-US'}, 'choice': [{'option

KeyError: 'meta'